In [2]:
!pip install nrel-pysam


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 14.1 MB/s eta 0:00:00


In [11]:
import numpy as np
import pandas as pd
from scipy.optimize import fsolve

def single_diode_model(irradiance, temperature):
    """
    Calculate the PV power generation using the single diode model.

    Parameters:
    - irradiance (float): Solar irradiance in W/m².
    - temperature (float): Cell temperature in Celsius.

    Returns:
    - power (float): Generated power in kW.
    """

    # Constants for the single diode model
    k = 1.38e-23  # Boltzmann's constant, J/K
    q = 1.6e-19  # Electron charge, C
    T_ref = 25 + 273.15  # Reference temperature in Kelvin
    G_ref = 1000  # Reference irradiance in W/m²
    I_sc = 5.0  # Short circuit current, A
    V_oc = 0.6  # Open circuit voltage, V
    n = 1.3  # Ideality factor
    R_s = 0.001  # Series resistance, ohms
    R_sh = 100.0  # Shunt resistance, ohms

    # Convert temperature to Kelvin
    T = temperature + 273.15

    # Thermal voltage
    V_t = k * T / q

    # Calculate the photocurrent I_ph
    I_ph = I_sc * (irradiance / G_ref)

    # Reverse saturation current I_0
    I_0 = I_sc / (np.exp(V_oc / (n * V_t)) - 1)

    # Voltage across the PV cell (assume max power point voltage is 80% of open circuit voltage)
    V = 0.8 * V_oc

    # Define the function for which we need to find the root (current I)
    def diode_equation(I):
        return I_ph - I_0 * (np.exp((V + I * R_s) / (n * V_t)) - 1) - (V + I * R_s) / R_sh - I

    # Solve for the current I using fsolve
    I_initial_guess = I_ph  # Initial guess for I
    I_solution = fsolve(diode_equation, I_initial_guess)[0]

    # Calculate power output (Power = Voltage * Current)
    power = V * I_solution / 1000  # Convert to kW

    return power

def generate_pv_power_data_from_csv(csv_file_path):
    """
    Generate synthetic PV power generation data based on user's choice of model (empirical or single diode),
    using input data from a CSV file. The function outputs an hourly time series of PV power generation.

    Parameters:
    - csv_file_path (str): Path to the CSV file containing the input data.

    Outputs:
    - synthetic_power_df (DataFrame): DataFrame containing the generated PV power data.
    - Saves the generated data to an Excel file.
    """

    # Step 1: Load the data from CSV
    try:
        df = pd.read_csv(csv_file_path, delimiter=';')
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{csv_file_path}' was not found.")
    except pd.errors.EmptyDataError:
        raise ValueError("The CSV file is empty.")
    except Exception as e:
        raise ValueError(f"An error occurred while loading the CSV file: {e}")

    # Optionally, handle missing values by filling with forward fill method
    df['latitude'].fillna(method='ffill', inplace=True)  # Forward fill as an example
    df['longitude'].fillna(method='ffill', inplace=True)

    # Extract datetime, temperature, and direct radiation from row 4 onwards (index 3 onwards)
    try:
        datetime_series = pd.to_datetime(df.iloc[3:, 0])
        temperature_series = df.iloc[3:, 1].astype(float)
        radiation_series = df.iloc[3:, 6].astype(float)
    except Exception as e:
        raise ValueError(f"An error occurred while processing the data columns: {e}")

    # Ensure data is valid
    if temperature_series.isnull().any() or radiation_series.isnull().any():
        raise ValueError("The CSV file contains missing values in temperature or radiation columns.")
    if (radiation_series < 0).any():
        raise ValueError("Radiation values should not be negative.")

    # Step 2: Choose the model
    model_input = input("Select the model (enter '0' for empirical or '1' for single diode): ").strip()

    # Convert numeric input to model type
    if model_input == '0':
        model = 'empirical'
    elif model_input == '1':
        model = 'single diode'
    else:
        raise ValueError("Invalid input. Please enter '0' for empirical or '1' for single diode.")

    # Step 3: Model-specific inputs and calculations
    if model == 'empirical':
        efficiency = 0.15
        temp_coefficient = 0.005
        ref_temp = 25

        # Calculate power based on the empirical model
        synthetic_power = efficiency * radiation_series * (1 - temp_coefficient * (temperature_series - ref_temp))

    elif model == 'single diode':
        synthetic_power = []
        for irradiance, temp in zip(radiation_series, temperature_series):
            power = single_diode_model(irradiance, temp)
            synthetic_power.append(power)
        synthetic_power = np.array(synthetic_power)

    # Step 4: Create DataFrame to hold the results
    synthetic_power_df = pd.DataFrame({
        'Time': datetime_series.reset_index(drop=True),
        'Irradiance (W/m²)': radiation_series.reset_index(drop=True),
        'Temperature (°C)': temperature_series.reset_index(drop=True),
        'Power Generation (kW)': synthetic_power
    })

    # Step 5: Save the results to an Excel file
    excel_filename = f"synthetic_pv_power_{model}.xlsx"
    try:
        synthetic_power_df.to_excel(excel_filename, index=False)
        print(f"Synthetic PV power data has been saved to {excel_filename}")
    except IOError:
        raise IOError(f"Failed to save the file: {excel_filename}")

    # Return the DataFrame
    return synthetic_power_df



# Example of calling the function with a given CSV file path
csv_file_path = '/content/drive/MyDrive/open-meteo-41.16N8.63W92m.csv'
synthetic_power_data = generate_pv_power_data_from_csv(csv_file_path)


<ipython-input-11-5c0610d76871>:80: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['latitude'].fillna(method='ffill', inplace=True)  # Forward fill as an example
<ipython-input-11-5c0610d76871>:81: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['longitude'].fillna(method='ffill', inplace=True)


Select the model (enter '0' for empirical or '1' for single diode): 0
Synthetic PV power data has been saved to synthetic_pv_power_empirical.xlsx
